In [ ]:
!pip install --upgrade google-generativeai openpyxl pandas tabulate

In [ ]:
import pandas as pd
import sqlite3
import google.generativeai as genai
from tabulate import tabulate

In [2]:

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [3]:
ad_sales = pd.read_excel("Product-Level Ad Sales and Metrics (mapped).xlsx")
total_sales = pd.read_excel("Product-Level Total Sales and Metrics (mapped).xlsx")
eligibility = pd.read_excel("Product-Level Eligibility Table (mapped).xlsx")

In [4]:
conn = sqlite3.connect(":memory:")
ad_sales.to_sql("ad_sales", conn, if_exists="replace", index=False)
total_sales.to_sql("total_sales", conn, if_exists="replace", index=False)
eligibility.to_sql("eligibility", conn, if_exists="replace", index=False)


4381

In [6]:
print("\nAd Sales Metrics:\n", ad_sales.head())
print("\nTotal Sales Metrics:\n", total_sales.head())
print("\nEligibility Table:\n", eligibility.head())



Ad Sales Metrics:
         date  item_id  ad_sales  impressions  ad_spend  clicks  units_sold
0 2025-06-01        0    332.96         1963     16.87       8           3
1 2025-06-01        1      0.00         1764     20.39      11           0
2 2025-06-01        2     95.99          169      0.48       0           1
3 2025-06-01        3   1001.93         6943     75.69      31           9
4 2025-06-01        4   1096.98        59046    401.39     285           5

Total Sales Metrics:
         date  item_id  total_sales  total_units_ordered
0 2025-06-01        0       309.99                    1
1 2025-06-01        3       338.00                    2
2 2025-06-01        4       617.99                    3
3 2025-06-01        9       219.00                    1
4 2025-06-01       12       534.00                    3

Eligibility Table:
   eligibility_datetime_utc  item_id  eligibility  \
0  2025-06-04 08:50:07.115       29        False   
1  2025-06-04 08:50:07.115      270         Tr

In [7]:
def get_schema():
    def table_info(df, name):
        return f"TABLE {name} ({', '.join(f'{col} {str(dt)}' for col,dt in zip(df.columns, df.dtypes))})"
    return "\n".join([table_info(ad_sales,'ad_sales'), table_info(total_sales,'total_sales'), table_info(eligibility,'eligibility')])

In [8]:
def nl2sql(question: str) -> str:
    schema = get_schema()
    prompt = f"""You are a data assistant. Given the tables and the user question, write a single SQLite SQL query to answer it.
Tables:
{schema}

Question: {question}

Reply with only a valid SQL query. Do not explain. Do not use LIMIT unless the question asks for a sample.
"""
    response = model.generate_content(prompt)
    # Remove formatting if Gemini wraps in ```
    sql = response.text.strip().split('\n')
    for s in sql:
        if s.strip().lower().startswith("select"):
            return s.strip()
    return response.text.strip()


In [9]:
def run_sql(sql):
    try:
        df = pd.read_sql_query(sql, conn)
        return df
    except Exception as e:
        return f"SQL error: {e}"

def pretty(df):
    if isinstance(df, str):
        return df
    if isinstance(df, pd.DataFrame) and df.shape == (1, 1):
        v = df.iloc[0, 0]
        # Formatting for float, int, or None/NaN
        if v is None or (isinstance(v, float) and pd.isnull(v)):
            return "No data."
        return f"**{v:,.2f}**" if isinstance(v, (int, float)) else str(v)
    return tabulate(df, headers='keys', tablefmt='github', showindex=False)

In [10]:
def ask(question):
    print(f"\nUser: {question}\n")
    sql = nl2sql(question)
    print(f"Generated SQL:\n{sql}\n")
    result = run_sql(sql)
    print("Answer:")
    print(pretty(result))
    return result


In [12]:
# 1. What is my total sales?
ask("What is my total sales?")






User: What is my total sales?

Generated SQL:
SELECT SUM(total_sales) FROM total_sales;

Answer:
**1,004,904.56**


,SUM(total_sales)
0,1004904.56


In [13]:
# 2. Calculate the RoAS (Return on Ad Spend).
ask("Calculate the RoAS (Return on Ad Spend).")






User: Calculate the RoAS (Return on Ad Spend).

Generated SQL:
SELECT CAST(SUM(ad_sales) AS REAL) / SUM(ad_spend) FROM ad_sales

Answer:
**7.92**


,CAST(SUM(ad_sales) AS REAL) / SUM(ad_spend)
0,7.915767


In [14]:
# 3. Which product had the highest CPC (Cost Per Click)?
ask("Which product had the highest CPC (Cost Per Click)?")


User: Which product had the highest CPC (Cost Per Click)?

Generated SQL:
SELECT item_id FROM ad_sales WHERE clicks > 0 ORDER BY CAST(ad_spend AS REAL) / clicks DESC LIMIT 1

Answer:
22


,item_id
0,22
